In [ ]:
# Import libraries
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import netCDF4
import matplotlib.pyplot as plt
import rioxarray
import rasterio.features

In [ ]:
# Open dataset
ds_1994_2023 = xr.open_dataset(r'1994_2023.nc')

In [ ]:
# Display dataset
ds_1994_2023

In [ ]:
# Alter 'time' from UTC to Vanuatu time (UTC + 11 hours)
ds_1994_2023['time'] = ds_1994_2023['time'] + np.timedelta64(11, 'h')

In [ ]:
# Calculate the new variable BGT
BGT = (1.33 * ds_1994_2023['temperature']) - (2.65 * np.sqrt(ds_1994_2023['temperature'])) + (3.21 * np.log10(ds_1994_2023['solar_radiation'] + 1)) + 3.5

# Add BGT to the dataset
ds_1994_2023['BGT'] = BGT

# Re-assign names to the variables for the calculation
RH = ds_1994_2023['relative_humidity']
WS = ds_1994_2023['wind_speed']
BGT = ds_1994_2023['BGT']

# Calculate the different equations needed to calculate the HLI based on the BGT
HLI_high = 8.62 + (0.38 * RH) + (1.55 * BGT) + np.exp(-WS + 2.4) - (0.5 * WS)
HLI_low = 10.66 + (0.28 * RH) + (1.30 * BGT) - WS
BGT_bl = 1 / (1 + (np.exp(-BGT - 25) / 2.25))
HLI_bl = BGT_bl * HLI_high + (1 - BGT_bl) * HLI_low

# Define a function to calculate the final HLI based on BGT conditions
def calculate_final_HLI(BGT, HLI_high, HLI_low, HLI_bl):
    return xr.where(BGT > 25, HLI_high, xr.where(BGT < 25, HLI_low, HLI_bl))

# Calculate the final HLI
final_HLI = calculate_final_HLI(BGT, HLI_high, HLI_low, HLI_bl)

# Add the final HLI to the dataset
ds_1994_2023['HLI'] = final_HLI

# Take the HLI to calculate HL balance
HLI = ds_1994_2023['HLI']

# Define threshold HLI values
threshold_high = 93
threshold_low = 77

# Calculate HL balance
HL_balance = xr.where(HLI > threshold_high,
                      HLI - threshold_high,
                      xr.where(HLI < threshold_low,
                               HLI - threshold_low,
                               0))

# Initialize AHL with zeros
AHL = xr.zeros_like(HL_balance)

# Perform cumulative sum with clipping to zero
for i in range(1, len(AHL.time)):
    # Calculate cumulative sum, ensuring no negative values
    AHL[i] = xr.where(AHL[i-1] + HL_balance[i] < 0, 0, AHL[i-1] + HL_balance[i])

# Add HL_balance and AHL to the dataset
ds_1994_2023['HL_balance'] = HL_balance
ds_1994_2023['AHL'] = AHL

In [ ]:
# Display dataset with HLI and AHL variables
ds_1994_2023

In [ ]:
# Download the provinces shapefile
provinces = gpd.read_file(r'vanuatu_provinces.shp')

# Clean Provinces Data
# Columns to delete
columns_to_delete = ['ADM1_PCODE', 'ADM1_REF', 'ADM1ALT1EN', 'ADM1ALT2EN', 'ADM0_EN', 'ADM0_PCODE', 'date', 'validOn', 'validTo']

# Delete the columns
provinces = provinces.drop(columns=columns_to_delete)

# Rename the column 
provinces = provinces.rename(columns={'ADM1_EN' : 'Province'})

# Set CRS to EPSG:4326
provinces = provinces.to_crs('EPSG:4326')

In [ ]:
# Display provinces dataset
provinces

In [ ]:
# Mask the dataset to include land cells only

# Rasterise the shapefile
shapes = [(geom, value) for geom, value in zip(provinces.geometry, provinces.index + 1)]
out_shape = (len(ds_1994_2023.latitude), len(ds_1994_2023.longitude))
transform = ds_1994_2023.rio.transform()

province_raster = rasterio.features.rasterize(
    shapes,
    out_shape=out_shape,
    transform=transform,
    fill=0,
    all_touched=True,
    dtype='int32'
)

# Assign the rasterised province IDs to a new DataArray
province_ids = xr.DataArray(
    province_raster,
    coords=[ds_1994_2023.latitude, ds_1994_2023.longitude],
    dims=["latitude", "longitude"]
)

# Add the province IDs to the dataset
ds_1994_2023["province_id"] = province_ids

# Create a mapping from province IDs to province names
province_mapping = {i + 1: name for i, name in enumerate(provinces['Province'])}

# Create a new variable for province names
ds_1994_2023['province_name'] = xr.DataArray(
    np.vectorize(province_mapping.get)(ds_1994_2023['province_id'].data),
    coords=ds_1994_2023['province_id'].coords,
    dims=ds_1994_2023['province_id'].dims
)

# Display dataset
ds_1994_2023

In [ ]:
# Plot the rasterized province IDs
plt.figure(figsize=(10, 6))
province_ids_plot = ds_1994_2023['province_id'].plot()
plt.title('Rasterized Province IDs with Shapefile Overlay')

# Overlay the shapefile
provinces.boundary.plot(ax=province_ids_plot.axes, color='red', linewidth=1)
plt.show()


In [ ]:
# Ensure consisitency throughout the dataset
# Convert 'province_name' to a fixed-length string type
ds_1994_2023['province_name'] = ds_1994_2023['province_name'].astype('str')

# Convert province_id to float32
ds_1994_2023['province_id'] = ds_1994_2023['province_id'].astype('float32')

In [ ]:
# Save dataset (if possible, dataset maybe too large depending on the computing system)
ds_1994_2023.to_netcdf(r'dataset.nc')

In [ ]:
# Take HLI and Province Name out of the xarray dataset and put it into a pandas dataset so that its easier to work with
HLI_data = ds_1994_2023[['HLI', 'province_name']].to_dataframe().reset_index()

In [ ]:
# Convert 'time' to datetime
HLI_data['time'] = pd.to_datetime(HLI_data['time'])

# Set 'time' as the index for resampling
HLI_data.set_index('time', inplace=True)

# Calculate daily maximum HLI for each grid cell
daily_max_hli_df = HLI_data.groupby(['latitude', 'longitude', 'province_name'])['HLI'].resample('D').max().reset_index()

# Display dataset
daily_max_hli_df

In [ ]:
# Calculate daily maximum HLI for each grid cell
daily_min_hli_df = HLI_data.groupby(['latitude', 'longitude', 'province_name'])['HLI'].resample('D').min().reset_index()

# Display dataset
daily_min_hli_df

In [ ]:
# Convert HLI_data and daily_max_hli_df to xarray dataset to save as a netcdf file
Save_HLI_data = HLI_data.to_xarray()
Save_HLI_data.to_netcdf(r'HLI_data.nc')

D_Max_HLI = daily_max_hli_df.to_xarray()
D_Max_HLI.to_netcdf(r'D_Max_HLI.nc')


D_Min_HLI = daily_min_hli_df.to_xarray()
D_Min_HLI.to_netcdf(r'D_Min_HLI.nc')

In [ ]:
#Take AHL and Province Name out of the xarray dataset and put it into a pandas dataset so that its easier to work with
AHL_data = ds_1994_2023[['AHL', 'province_name']].to_dataframe().reset_index()

# Convert 'time' to datetime
AHL_data['time'] = pd.to_datetime(AHL_data['time'])

# Set 'time' as the index for resampling
AHL_data.set_index('time', inplace=True)

# Calculate daily maximum AHL for each grid cell across all years
daily_max_ahl_df = AHL_data.groupby(['latitude', 'longitude', 'province_name'])['AHL'].resample('D').max().reset_index()

daily_max_ahl_df

In [ ]:
# Convert AHL_data and daily_max_ahl_df to xarray dataset to save as a netcdf file
Save_AHL_data = AHL_data.to_xarray()
Save_AHL_data.to_netcdf(r'AHL_data.nc')

D_Max_AHL = daily_max_ahl_df.to_xarray()
D_Max_AHL.to_netcdf(r'D_Max_AHL.nc')